## Import Libraries

In [26]:
import time
import rospy
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from dual_quaternion import Quaternion
from dual_quaternion import DualQuaternion
from dual_quaternion import plot_states_quaternion, plot_states_position, fancy_plots_4, fancy_plots_1, plot_norm_quat, plot_angular_velocities, plot_linear_velocities, fancy_plots_3, plot_norm_real, plot_norm_dual
from scipy.spatial.transform import Rotation as R
from nav_msgs.msg import Odometry
from casadi import Function

## Working with numeric Entries

In [27]:

# Defining of the vectors using casadi
theta1 = ca.SX([np.pi/2])
n1 = ca.SX([0.0, 0.0, 1.0])
q1 = ca.vertcat(ca.cos(theta1/2), ca.sin(theta1/2)@n1)
t1 = ca.SX([0.0, 1.0, 1.0, 0.0])

# Defining of the vectors using casadi
theta2 = ca.SX([ca.pi/4])
n2 = ca.SX([1.0, 0.0, 0.0])
q2 = ca.vertcat(ca.cos(theta2/2), ca.sin(theta2/2)@n2)
t2 = ca.SX([0.0, 0.0, 2.0, 0.0])


# Init Dualquaternion
Q1 = DualQuaternion.from_pose(quat = q1, trans = t1)
Q2 = DualQuaternion.from_pose(quat = q2, trans = t2)

In [28]:
Q1.get_trans.get[:, 0]

SX(@1=0, @2=1, [@1, @2, @2, @1])

In [29]:
Q1.get_trans

Quaternion: @1=0, @2=1, [@1, @2, @2, @1]

In [30]:
Q1.get

SX(@1=0, @2=0.707107, [0.707107, @1, @1, @2, @1, @2, 5.55112e-17, @1])

In [31]:
Q1.get_quat.get[:, 0]

SX(@1=0, [0.707107, @1, @1, 0.707107])

In [32]:
## Computing whole transformation
Q3  = Q1 * Q2

In [33]:
Q3.get[:, 0]

SX(@1=0.653281, @2=0, [@1, 0.270598, 0.270598, 0.653281, @2, @2, @1, -0.270598])

## Working with Symbolic Variables

In [34]:
# Defining First transformation
theta_1 = ca.MX.sym('theta_1', 1, 1)
n_1 = ca.MX([0.0, 0.0, 1.0])
q_1 = ca.vertcat(ca.cos(theta_1/2), ca.sin(theta_1/2)@n_1)
tx_1 = ca.MX.sym("x_1", 1, 1)
ty_1 = ca.MX.sym("y_1", 1, 1)
t_1 = ca.vertcat(0.0, tx_1, ty_1, 0.0)

# Defining second transformation
theta_2 = ca.MX.sym('theta_2', 1, 1)
n_2 = ca.MX([1.0, 0.0, 0.0])
q_2 = ca.vertcat(ca.cos(theta_2/2), ca.sin(theta_2/2)@n_2)
tx_2 = ca.MX.sym("x_2", 1, 1)
ty_2 = ca.MX.sym("y_2", 1, 1)
t_2 = ca.vertcat(0.0, 0.0, ty_2, 0.0)
aux_symbolic_total = ca.vertcat(theta_1, tx_1, ty_1, theta_2, ty_2)
aux_symbolic = ca.vertcat(theta_1, tx_1, ty_1)

In [35]:
Q1 = DualQuaternion.from_pose(quat = q_1, trans = t_1)
Q2 = DualQuaternion.from_pose(quat = q_2, trans = t_2)

In [36]:
Q1.get

MX(@1=cos((theta_1/2)), @2=([0, 0, 1]*sin((theta_1/2))), @3=vertcat(0, x_1, y_1, 0), @4=@2[2], vertcat(@1, @2, (0.5*vertcat((-dot(@3[1:4], @2)), ((@1*@3[1:4])+vertcat((y_1*@4), (-(x_1*@4)), ((x_1*@2[1])-(y_1*@2[0]))))))))

In [37]:
Q1.get_trans.get[:, 0]

MX(@1=vertcat(0, x_1, y_1, 0), @2=([0, 0, 1]*sin((theta_1/2))), @3=cos((theta_1/2)), @4=@2[2], @5=(2.*(0.5*vertcat((-dot(@1[1:4], @2)), ((@3*@1[1:4])+vertcat((y_1*@4), (-(x_1*@4)), ((x_1*@2[1])-(y_1*@2[0]))))))), @6=@2[0], @7=@2[1], @8=@2[2], @9=vertcat(@3, (-@6), (-@7), (-@8)), @10=@5[3], @11=@5[2], @12=@5[1], vertcat(((@5[0]*@3)-dot(@5[1:4], @9[1:4])), (((@5[0]*@9[1:4])+(@3*@5[1:4]))+vertcat(((@10*@7)-(@11*@8)), ((@12*@8)-(@10*@6)), ((@11*@6)-(@12*@7))))))

In [38]:
aux_symbolic

MX(vertcat(theta_1, x_1, y_1))

In [39]:
## Computing whole transformation
Q3  = Q1 * Q2

### Computing Functions Based on CasADi

In [40]:
Function_DualQuat_from_Pose = Function('DualQuat_from_Pose', [aux_symbolic], [Q1.get[:, 0]])
Function_DualQuat_get_trans = Function('DualQuat_get_trans', [aux_symbolic], [Q1.get_trans.get[:, 0]])
Function_DualQuat_get_quat = Function('DualQuat_get_quat', [aux_symbolic], [Q1.get_quat.get[:, 0]])
Function_DualQuat_get_trans_t = Function('DualQuat_get_trans_t', [aux_symbolic_total], [Q3.get_trans.get[:, 0]])
Function_DualQuat_get_quat_t = Function('DualQuat_get_quat_t', [aux_symbolic_total], [Q3.get_quat.get[:, 0]])

In [41]:
input = np.array([np.pi/2, 1, 1])
input_t = np.array([np.pi/2, 1, 1, np.pi/4, 2])

In [42]:
Dual = Function_DualQuat_from_Pose(input)

In [43]:
Dual

DM([0.707107, 0, 0, 0.707107, -0, 0.707107, 5.55112e-17, 0])

In [44]:
trans = Function_DualQuat_get_trans(input)

In [45]:
trans

DM([-0, 1, 1, 0])

In [46]:
quat = Function_DualQuat_get_quat(input)

In [47]:
quat

DM([0.707107, 0, 0, 0.707107])

In [48]:
trans_t = Function_DualQuat_get_trans_t(input_t)

In [49]:
trans_t

DM([0, -1, 1, 0])

In [50]:
quat_t = Function_DualQuat_get_quat_t(input_t)

In [51]:
quat_t

DM([0.653281, 0.270598, 0.270598, 0.653281])